# Solving Laplace's Equation on an Annulus using PINNs with DeepXDE

This notebook demonstrates how to use Physics-Informed Neural Networks (PINNs) with DeepXDE to solve Laplace's equation (Δu = 0) on an annular domain.

**Laplace's Equation:** Δu(x, y) = 0

**Annulus Domain:** The domain is an annulus centered at the origin [0,0] with an inner radius of 1 and an outer radius of 2.

**Boundary Conditions:**
1. Outer boundary (r=2): u = 0 (Dirichlet)
2. Inner boundary (r=1): u = sin(θ) (Dirichlet, angle-dependent)

## 1. Setup: Import Libraries

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

## 2. Geometry Definition

The annulus is defined by an outer disk (radius 2) and an inner disk (radius 1), both centered at the origin. The annulus is the region between these two circles.

In [ ]:
center = [0, 0]
r_outer = 2.0
r_inner = 1.0

disk_outer = dde.geometry.Disk(center, r_outer)
disk_inner = dde.geometry.Disk(center, r_inner)

# Define the annulus using CSG (Constructive Solid Geometry) difference
geom = disk_outer - disk_inner

# Optional: Plot geometry to verify
fig, ax = plt.subplots(figsize=(6,6))
ax.add_patch(plt.Circle(center, r_outer, color='lightblue', alpha=0.7, label='Outer Disk Region'))
ax.add_patch(plt.Circle(center, r_inner, color='white', label='Inner Disk Region (Removed)'))
ax.plot(*dde.geometry.Disk(center, r_outer).boundary_points().T, 'b-', label='Outer Boundary (r=2)')
ax.plot(*dde.geometry.Disk(center, r_inner).boundary_points().T, 'r-', label='Inner Boundary (r=1)')
ax.set_xlim([-r_outer-0.5, r_outer+0.5])
ax.set_ylim([-r_outer-0.5, r_outer+0.5])
ax.set_aspect('equal', adjustable='box')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Annulus Domain Geometry')
plt.legend()
plt.grid(True)
plt.show()

## 3. PDE and Boundary Condition Definition

**PDE:** Δu = 0 (Laplacian of u is zero)

**Boundary Conditions:**
- Outer (r=2): u = 0
- Inner (r=1): u = sin(θ)

In [ ]:
def pde(x, y_pred):
    """Defines Laplace's equation: dy_xx + dy_yy = 0"""
    dy_xx = dde.grad.hessian(y_pred, x, i=0, j=0)
    dy_yy = dde.grad.hessian(y_pred, x, i=1, j=1)
    return dy_xx + dy_yy

# Outer Boundary (r=2, u=0)
def boundary_outer(x_coords, on_boundary_annulus):
    # x_coords are points on the boundary of the annulus.
    # We need to select those that are on the outer circle (r=2).
    return np.isclose(np.linalg.norm(x_coords, axis=-1), r_outer)

def func_outer_val(x_coords):
    return np.zeros((x_coords.shape[0], 1), dtype=dde.config.real(np))

bc_outer = dde.DirichletBC(geom, func_outer_val, boundary_outer)

# Inner Boundary (r=1, u=sin(theta))
def boundary_inner(x_coords, on_boundary_annulus):
    # Select points on the inner circle (r=1).
    return np.isclose(np.linalg.norm(x_coords, axis=-1), r_inner)

def func_inner_val(x_coords):
    # x_coords is a batch: x_coords[:, 0] is x, x_coords[:, 1] is y
    cart_x = x_coords[:, 0:1]
    cart_y = x_coords[:, 1:2]
    theta = np.arctan2(cart_y, cart_x) # theta is in [-pi, pi]
    return np.sin(theta)

bc_inner = dde.DirichletBC(geom, func_inner_val, boundary_inner)

# Combine boundary conditions
bcs = [bc_outer, bc_inner]

## 4. Problem Setup (Data)

Define the PDE problem using `dde.data.PDE`, specifying geometry, PDE, BCs, and training/test points.

In [ ]:
# Number of training points
N_domain = 3000    # Points inside the domain
N_boundary = 1500  # Points on both boundaries (DeepXDE distributes these)

# Number of test points for validation
N_test = 1000

data = dde.data.PDE(
    geom,
    pde,
    bcs, # List of boundary condition objects
    num_domain=N_domain,
    num_boundary=N_boundary,
    num_test=N_test,
    solution=None # No analytical solution for testing this specific case easily
)

## 5. Network and Model Creation

A Feed-forward Neural Network (FNN) approximates the solution u(x,y).

In [ ]:
# Network parameters
input_dim = 2  # (x, y)
output_dim = 1 # u(x,y)
num_hidden_layers = 4
neurons_per_layer = 64 # Slightly increased neurons for potentially more complex solution
activation_fn = "tanh"
kernel_initializer = "Glorot uniform"

net = dde.maps.FNN(
    [input_dim] + [neurons_per_layer] * num_hidden_layers + [output_dim],
    activation_fn,
    kernel_initializer
)

# Create the DeepXDE model
model = dde.Model(data, net)

## 6. Model Training

Compile the model and train it.

In [ ]:
# Compile the model
learning_rate = 1e-3
# Loss weights can be used to balance PDE and BC losses if needed, e.g., [1, 10, 10]
model.compile("adam", lr=learning_rate)

# Train the model
num_iterations = 25000 # Adjust based on loss behavior
print(f"Starting training for {num_iterations} iterations...")
losshistory, train_state = model.train(iterations=num_iterations, display_every=1000)
print("Training finished.")

## 7. Results and Visualization

Visualize training progress (loss history) and the predicted solution.

In [ ]:
# Plot loss history
dde.utils.plot_loss_history(losshistory)
plt.title('Loss History for Annulus Problem (Log Scale)')
plt.show()

# Plot the predicted solution
plot_resolution = 100
x_lin = np.linspace(-r_outer, r_outer, plot_resolution)
y_lin = np.linspace(-r_outer, r_outer, plot_resolution)
X_plot, Y_plot = np.meshgrid(x_lin, y_lin)
plot_points_flat = np.vstack((X_plot.ravel(), Y_plot.ravel())).T

# Filter points to be within the annulus
is_inside_domain = geom.inside(plot_points_flat)
plot_points_inside_annulus = plot_points_flat[is_inside_domain]

if plot_points_inside_annulus.shape[0] > 0:
    u_pred_annulus = model.predict(plot_points_inside_annulus)
    
    plt.figure(figsize=(8, 7))
    contour = plt.tricontourf(
        plot_points_inside_annulus[:, 0],
        plot_points_inside_annulus[:, 1],
        u_pred_annulus.ravel(),
        levels=100,
        cmap='viridis'
    )
    plt.colorbar(contour, label='Predicted u(x,y)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Predicted Solution u(x,y) on Annulus Domain')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    
    # Add circles for boundaries for visual clarity
    circle_inner_plot = plt.Circle(center, r_inner, color='black', fill=False, linestyle='--', linewidth=1.0)
    circle_outer_plot = plt.Circle(center, r_outer, color='black', fill=False, linestyle='--', linewidth=1.0)
    ax.add_artist(circle_inner_plot)
    ax.add_artist(circle_outer_plot)
else:
    plt.figure(figsize=(8, 7))
    plt.text(0.5, 0.5, "No points found inside annulus for plotting", 
             horizontalalignment='center', verticalalignment='center', 
             transform=plt.gca().transAxes)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Predicted Solution u(x,y) on Annulus Domain')

plt.show()

## 8. Conclusion

This notebook demonstrated solving Laplace's equation on an annulus with angle-dependent Dirichlet boundary conditions using DeepXDE.

Key steps included:
1. Defining the annulus geometry using CSG operations.
2. Setting up Laplace's PDE and two distinct Dirichlet boundary conditions (one constant, one angle-dependent).
3. Configuring the PINN model (network architecture, data sampling).
4. Training the model.
5. Visualizing the loss history and the 2D solution plot.

The known analytical solution for Laplace's equation in an annulus with u(r_inner, θ) = sin(θ) and u(r_outer, θ) = 0 is of the form u(r,θ) = (A*r + B/r)sin(θ). The constants A and B can be found by applying the boundary conditions:
- At r = r_inner = 1:  (A*1 + B/1)sin(θ) = sin(θ)  => A + B = 1
- At r = r_outer = 2:  (A*2 + B/2)sin(θ) = 0      => 2A + B/2 = 0 => 4A + B = 0

Solving these: 
(4A + B) - (A + B) = 0 - 1 => 3A = -1 => A = -1/3
B = 1 - A = 1 - (-1/3) = 4/3

So, the analytical solution is u(r,θ) = (-1/3 * r + 4/3 / r)sin(θ).
This could be used for a more quantitative error analysis if desired.